## Exercícios

> Retirados de [learn-python: sqlalchemy_orm-questions](https://aviadr1.github.io/learn-advanced-python/11_db_access/exercise/sqlalchemy_orm-questions.html).

#### Q1.

Baixa e extraia o arquivo compactado com o banco de dados [Chinook database](https://www.sqlitetutorial.net/sqlite-sample-database/). Salve o arquivo `chinook.db` na mesma pasta deste script.
* Link para baixar: http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip

<img width=500 src=https://www.sqlitetutorial.net/wp-content/uploads/2015/11/sqlite-sample-database-color.jpg>


In [1]:
import requests
import zipfile
import os

# URL do arquivo ZIP do banco de dados Chinook
url = "http://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip"

# Nome do arquivo ZIP a ser baixado
zip_file_name = "chinook.zip"

# Baixar o arquivo ZIP
response = requests.get(url)
with open(zip_file_name, "wb") as file:
    file.write(response.content)

print("Arquivo ZIP baixado com sucesso.")

# Extrair o conteúdo do ZIP
with zipfile.ZipFile(zip_file_name, "r") as zip_ref:
    zip_ref.extractall(".")

print("Arquivo extraído com sucesso.")

# Verificar se o arquivo chinook.db foi salvo
if os.path.exists("Chinook_Sqlite_AutoIncrementPKs.sqlite"):
    os.rename("Chinook_Sqlite_AutoIncrementPKs.sqlite", "chinook.db")
    print("Banco de dados salvo como 'chinook.db'.")
else:
    print("O arquivo 'chinook.db' não foi encontrado.")


Arquivo ZIP baixado com sucesso.
Arquivo extraído com sucesso.
O arquivo 'chinook.db' não foi encontrado.


#### Q2.

Leia o código e os comentários das células a seguir para entender como acessamos os modelos ORM de um banco já existente.

In [ ]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.orm import Session

engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

### extrai as classes da base de dados Chinook
metadata = MetaData()
metadata.reflect(engine)

# O metadata tem informações sobre as tabelas
# que serão usadas para criar os modelos ORM
for table_name, table in metadata.tables.items():
    print(table_name)
    print(table.columns.keys())
    print(table.columns.items())
    print('-'*25)

### configura o objeto Base mapeando os modelos ORM das tabelas
from sqlalchemy.ext.automap import automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# o objeto Base tem os modelos ORM que podemos usar
# para manipular o banco de dados
print(Base.classes.items())

In [ ]:
# A seguir um exemplo de query na tabela Albums
# usamos o objeto Base para acessar cada modelo ORM.

session = Session(engine)
res = session.scalars(select(Base.classes.albums))
first_album = res.first()
print(first_album.AlbumId, first_album.Title)

#### Q3. 
Com base nos códigos anteriores realize as operações solicitadas nas células a seguir:


In [9]:
### Imprima os três primeiros registros da tabela tracks
from sqlalchemy import create_engine, MetaData, select
from sqlalchemy.orm import Session
from sqlalchemy.ext.automap import automap_base

# Configurar a engine para conectar ao banco chinook.db
engine = create_engine("sqlite+pysqlite:///chinook.db", echo=False)

# Carregar metadados e refletir tabelas existentes
metadata = MetaData()
metadata.reflect(engine)

# Configurar ORM automaticamente com automap_base
Base = automap_base(metadata=metadata)
Base.prepare()

# Criar uma sessão para realizar consultas
session = Session(engine)

# Query para selecionar registros da tabela 'tracks'
res = session.scalars(select(Base.classes.tracks))

# Imprimir os três primeiros registros
for i, track in enumerate(res, start=1):
    print(f"TrackId: {track.TrackId}, Name: {track.Name}, AlbumId: {track.AlbumId}, MediaTypeId: {track.MediaTypeId}, GenreId: {track.GenreId}")
    if i == 3:  # Parar após os três primeiros registros
        break

# Fechar a sessão
session.close()

TrackId: 1, Name: For Those About To Rock (We Salute You), AlbumId: 1, MediaTypeId: 1, GenreId: 1
TrackId: 2, Name: Balls to the Wall, AlbumId: 2, MediaTypeId: 2, GenreId: 1
TrackId: 3, Name: Fast As a Shark, AlbumId: 3, MediaTypeId: 2, GenreId: 1


In [10]:
### Imprima o nome da faixa e o título do álbum das primeiras 20 faixas na tabela tracks.
from sqlalchemy import select
from sqlalchemy.orm import Session

# Criar uma sessão para realizar consultas
session = Session(engine)

# Query para selecionar os registros da tabela 'tracks' e 'albums'
# Fazendo uma junção entre as tabelas tracks e albums para obter o título do álbum
res = session.execute(
    select(Base.classes.tracks.Name, Base.classes.albums.Title)
    .join(Base.classes.albums, Base.classes.tracks.AlbumId == Base.classes.albums.AlbumId)
    .limit(20)
)

# Imprimir o nome da faixa e o título do álbum
for track, album in res:
    print(f"Track Name: {track}, Album Title: {album}")

# Fechar a sessão
session.close()

Track Name: For Those About To Rock (We Salute You), Album Title: For Those About To Rock We Salute You
Track Name: Put The Finger On You, Album Title: For Those About To Rock We Salute You
Track Name: Let's Get It Up, Album Title: For Those About To Rock We Salute You
Track Name: Inject The Venom, Album Title: For Those About To Rock We Salute You
Track Name: Snowballed, Album Title: For Those About To Rock We Salute You
Track Name: Evil Walks, Album Title: For Those About To Rock We Salute You
Track Name: C.O.D., Album Title: For Those About To Rock We Salute You
Track Name: Breaking The Rules, Album Title: For Those About To Rock We Salute You
Track Name: Night Of The Long Knives, Album Title: For Those About To Rock We Salute You
Track Name: Spellbound, Album Title: For Those About To Rock We Salute You
Track Name: Balls to the Wall, Album Title: Balls to the Wall
Track Name: Fast As a Shark, Album Title: Restless and Wild
Track Name: Restless and Wild, Album Title: Restless and Wi

In [11]:
### Imprima as 10 primeiras vendas de faixas da tabela invoice_items
### Para essas 10 primeiras vendas, imprima os nomes das faixas vendidas e a quantidade vendida.
from sqlalchemy import select
from sqlalchemy.orm import Session

# Criar uma sessão para realizar consultas
session = Session(engine)

# Query para selecionar os registros da tabela 'invoice_items' e 'tracks'
# Fazendo uma junção entre as tabelas invoice_items e tracks para obter o nome da faixa
res = session.execute(
    select(Base.classes.invoice_items.Quantity, Base.classes.tracks.Name)
    .join(Base.classes.tracks, Base.classes.invoice_items.TrackId == Base.classes.tracks.TrackId)
    .limit(10)
)

# Imprimir os nomes das faixas e a quantidade vendida
for quantity, track_name in res:
    print(f"Track Name: {track_name}, Quantity Sold: {quantity}")

# Fechar a sessão
session.close()

Track Name: Balls to the Wall, Quantity Sold: 1
Track Name: Restless and Wild, Quantity Sold: 1
Track Name: Put The Finger On You, Quantity Sold: 1
Track Name: Inject The Venom, Quantity Sold: 1
Track Name: Evil Walks, Quantity Sold: 1
Track Name: Breaking The Rules, Quantity Sold: 1
Track Name: Dog Eat Dog, Quantity Sold: 1
Track Name: Overdose, Quantity Sold: 1
Track Name: Love In An Elevator, Quantity Sold: 1
Track Name: Janie's Got A Gun, Quantity Sold: 1


In [12]:
### Imprima os nomes das 10 faixas mais vendidas e quantas vezes foram vendidas.
from sqlalchemy import select, func
from sqlalchemy.orm import Session

# Criar uma sessão para realizar consultas
session = Session(engine)

# Query para selecionar as faixas mais vendidas, somando as quantidades vendidas
res = session.execute(
    select(Base.classes.tracks.Name, func.sum(Base.classes.invoice_items.Quantity).label('TotalSold'))
    .join(Base.classes.invoice_items, Base.classes.tracks.TrackId == Base.classes.invoice_items.TrackId)
    .group_by(Base.classes.tracks.Name)
    .order_by(func.sum(Base.classes.invoice_items.Quantity).desc())
    .limit(10)
)

# Imprimir o nome das faixas e a quantidade vendida
for track_name, total_sold in res:
    print(f"Track Name: {track_name}, Quantity Sold: {total_sold}")

# Fechar a sessão
session.close()

Track Name: The Trooper, Quantity Sold: 5
Track Name: Untitled, Quantity Sold: 4
Track Name: The Number Of The Beast, Quantity Sold: 4
Track Name: Sure Know Something, Quantity Sold: 4
Track Name: Hallowed Be Thy Name, Quantity Sold: 4
Track Name: Eruption, Quantity Sold: 4
Track Name: Where Eagles Dare, Quantity Sold: 3
Track Name: Welcome Home (Sanitarium), Quantity Sold: 3
Track Name: Sweetest Thing, Quantity Sold: 3
Track Name: Surrender, Quantity Sold: 3


In [13]:
### Quem são os 10 artistas que mais venderam?
### dica: você precisa juntar as tabelas invoice_items, tracks, albums e artists
from sqlalchemy import select, func
from sqlalchemy.orm import Session

# Criar uma sessão para realizar consultas
session = Session(engine)

# Query para selecionar os artistas que mais venderam, somando as quantidades de faixas vendidas
res = session.execute(
    select(Base.classes.artists.Name, func.sum(Base.classes.invoice_items.Quantity).label('TotalSold'))
    .join(Base.classes.tracks, Base.classes.invoice_items.TrackId == Base.classes.tracks.TrackId)
    .join(Base.classes.albums, Base.classes.tracks.AlbumId == Base.classes.albums.AlbumId)
    .join(Base.classes.artists, Base.classes.albums.ArtistId == Base.classes.artists.ArtistId)
    .group_by(Base.classes.artists.Name)
    .order_by(func.sum(Base.classes.invoice_items.Quantity).desc())
    .limit(10)
)

# Imprimir o nome do artista e a quantidade total vendida
for artist_name, total_sold in res:
    print(f"Artist Name: {artist_name}, Quantity Sold: {total_sold}")

# Fechar a sessão
session.close()

Artist Name: Iron Maiden, Quantity Sold: 140
Artist Name: U2, Quantity Sold: 107
Artist Name: Metallica, Quantity Sold: 91
Artist Name: Led Zeppelin, Quantity Sold: 87
Artist Name: Os Paralamas Do Sucesso, Quantity Sold: 45
Artist Name: Deep Purple, Quantity Sold: 44
Artist Name: Faith No More, Quantity Sold: 42
Artist Name: Lost, Quantity Sold: 41
Artist Name: Eric Clapton, Quantity Sold: 40
Artist Name: R.E.M., Quantity Sold: 39
